In [1]:
from model import nouns_extractor, keywords_sum_similarity, recommend_by_keyword, make_reserved_bookmark_list, get_category

In [3]:
def translate(req):
    
    title = req['title']
    description = req['description']

    if title == 'apiapiapi' and description == 'apiapiapi':
        x = '''
        설명 :
        10 + title 안의 명사의 갯수 만큼 길이의 dictionary가 return.
        배열 안의 원소는 2개의 배열로 이루어져 있으며 첫 번째에는 추천키워드, 두 번째에는 각각의 keyword와 description으로부터 계산된 similarity_sum이 들어간다.
        * 만약 추천키워드가 model 안에 없는 경우 맨 앞 단어에는 !가 붙고, 그 value = -1로 표기된다.
        ( ex. 리턴값에 백엔드가 단어로 들어왔는데 모델에 없는 경우..(받은 배열의 이름을 res라 함.) res[!백엔드] == -1 )
        '''
        # make_response(x)
        return x, 200

    # stttime = time.time()

    title_nouns = nouns_extractor(title)
    description_nouns = nouns_extractor(description)
    reserved_bookmark_list = make_reserved_bookmark_list(title_nouns)
    recommend_keywords = keywords_sum_similarity(reserved_bookmark_list,description_nouns)
    # 함수 개선의 필요성이 느껴진다.. 처음 시작은 약 4초, 그 후에는 함수/변수/모델이 모두 로드가 된 상태이기 때문에 약 0.5초정도 걸림.
    # (기본 pretrained_model 사용시..)
    
    category = get_category(['프로그래밍','날씨','하늘'])

    print(category)
    # print('running_time = ', time.time() - stttime)
    
    return recommend_keywords

In [2]:
from gensim.models import Word2Vec
import os
os.environ['JAVA_HOME'] = r'/usr/bin/jar'

In [10]:
model = Word2Vec.load('../model/pretrained_ko.bin') # 새로운 경로 지정.
model_input_words = tuple(model.wv.index2word)

In [11]:
model.wv.most_similar("한글")

[('우리말', 0.6856735944747925),
 ('훈민정음', 0.6746662259101868),
 ('한국어', 0.6672539710998535),
 ('로마자', 0.6571487188339233),
 ('영문', 0.6518102884292603),
 ('한문', 0.6470773220062256),
 ('글꼴', 0.6337352991104126),
 ('한자', 0.629895806312561),
 ('낱자', 0.6262421011924744),
 ('맞춤법', 0.6166303157806396)]

In [74]:
len(model.wv.index2word)

x = '경영, 정보/기술, 금융, 개발, 구인/구직, 건강, 환경, 뷰티, 여행, 맛집/카페, 자기계발, 음식/요리'.split(', ')
# it --> 정보, (인크루트) --> del, 
# 자기개발, 맛집, 

In [88]:
y=[]
for i in range(len(x)):
    if '/' in x[i]:
        y.append(tuple(x[i].split('/')))
    else:
        y.append(x[i])
print(x)
categories = [('경영'), ('정보', '기술'), ('금융'), ('개발'), ('구인', '구직'), ('건강'), ('환경'), ('뷰티'), ('여행'), ('식당', '카페'), ('자기','공부'),('음식', '요리')]
print(categories)

cate_dic = {categories[i]:x[i] for i in range(len(x))}
print(cate_dic)

['경영', '정보/기술', '금융', '개발', '구인/구직', '건강', '환경', '뷰티', '여행', '맛집/카페', '자기계발', '음식/요리']
['경영', ('정보', '기술'), '금융', '개발', ('구인', '구직'), '건강', '환경', '뷰티', '여행', ('식당', '카페'), ('자기', '공부'), ('음식', '요리')]
{'경영': '경영', ('정보', '기술'): '정보/기술', '금융': '금융', '개발': '개발', ('구인', '구직'): '구인/구직', '건강': '건강', '환경': '환경', '뷰티': '뷰티', '여행': '여행', ('식당', '카페'): '맛집/카페', ('자기', '공부'): '자기계발', ('음식', '요리'): '음식/요리'}


In [38]:
model.wv.most_similar(positive = ['구인','구직'])

[('취업', 0.561211347579956),
 ('연수', 0.5039308071136475),
 ('구직자', 0.5037397146224976),
 ('교육장', 0.49912774562835693),
 ('직업', 0.4980913996696472),
 ('사무원', 0.49557119607925415),
 ('임직원', 0.4732576608657837),
 ('사무직', 0.4702960252761841),
 ('알선', 0.46453529596328735),
 ('장학사', 0.46412405371665955)]

In [89]:
def get_category(hashtags):
    categories = [('경영'), ('정보', '기술'), ('금융'), ('개발'), ('구인', '구직'), ('건강'), ('환경'), ('뷰티'), ('여행'), ('식당', '카페'), ('자기','공부'),('음식', '요리')]
    cate_dic = {'경영': '경영', ('정보', '기술'): '정보/기술', '금융': '금융', '개발': '개발', ('구인', '구직'): '구인/구직', '건강': '건강', '환경': '환경', '뷰티': '뷰티', '여행': '여행', ('식당', '카페'): '맛집/카페', ('자기', '공부'): '자기계발', ('음식', '요리'): '음식/요리'}
    weights = [0]*len(categories)

    for i in range(len(categories)):
        weights[i] = model.wv.n_similarity(categories[i],hashtags)
    
    return cate_dic[categories[weights.index(max(weights))]]

In [91]:
get_category(['취업','대학생','구인'])

'구인/구직'

In [54]:
model.wv.n_similarity(['구인','사무원','구직'],['알선','임직원','연수'])

0.6609084

In [59]:
model.wv.most_similar('공부')

[('가르치', 0.6655912399291992),
 ('중퇴', 0.6448310613632202),
 ('수학', 0.6283489465713501),
 ('졸업', 0.5976732969284058),
 ('한학', 0.5947862267494202),
 ('수료', 0.5928074717521667),
 ('영문학', 0.5868287086486816),
 ('전공', 0.5848740339279175),
 ('배우', 0.5693216919898987),
 ('강론', 0.5641564726829529)]

In [62]:
z = {'qwer':0.1,'wer':0.3,'ssd':0.999,'vxcb':-1,'ghbv':333}

xxx = sorted(list(z.keys()), key = lambda x: -z[x])[:3]
xxx

['ghbv', 'ssd', 'wer']

## !  핫픽스.. 모델이 기존 단어에 추가된 훈련이 아님...,,,, 